# Example

## Bands WorkGraph

In [11]:
from workgraph_collections.qe.bands import bands_workgraph
from aiida import load_profile
load_profile()# -*- coding: utf-8 -*-
"""PdosWorkGraph."""

from aiida import orm
from aiida_workgraph import WorkGraph
from aiida_workgraph.decorator import node
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.calculations.dos import DosCalculation
from aiida_quantumespresso.calculations.projwfc import ProjwfcCalculation
from aiida_wannier90_workflows.workflows.base.wannier90 import Wannier90BaseWorkChain
from aiida_wannier90_workflows.workflows.base.projwfc import ProjwfcBaseWorkChain
from aiida_wannier90_workflows.workflows.base.pw2wannier90 import Pw2wannier90BaseWorkChain

@node.calcfunction()
def prepare_wannier90_pp_inputs(parameters, output_parameters):
    """Prepare the inputs of wannier90 calculation before submission.
    """
    parameters = parameters.get_dict()
    parameters["fermi_energy"] = output_parameters.get_dict().get('fermi_energy')
    return orm.Dict(parameters)

@node.calcfunction()
def prepare_wannier90_inputs(parameters, output_parameters):
    """Prepare the inputs of wannier90 calculation.
    """
    parameters = parameters.get_dict()
    parameters["fermi_energy"] = output_parameters.get_dict().get('fermi_energy')
    return orm.Dict(parameters)    

@node.graph_builder()
def wannier_workgraph(structure=None, inputs=None):
    """Generate PdosWorkGraph."""
    inputs = {} if inputs is None else inputs
    # create workgraph
    wg = WorkGraph()
    wg.ctx = {'current_structure': structure, 'current_number_of_bands': None, 'parent_folder': None}
    # -------- scf -----------
    scf_node = wg.nodes.new(PwBaseWorkChain, name='scf')
    scf_inputs = inputs.get('scf', {})
    scf_inputs['pw.structure'] = structure
    scf_node.set(scf_inputs)
    # -------- nscf -----------
    nscf_node = wg.nodes.new(PwBaseWorkChain, name='nscf')
    nscf_inputs = inputs.get('nscf', {})
    nscf_inputs.update({'pw.structure': structure,
                        'pw.parent_folder': scf_node.outputs['remote_folder']
    })
    nscf_node.set(nscf_inputs)
    # -------- projwfc -----------
    projwfc = wg.nodes.new(ProjwfcBaseWorkChain, name='projwfc',
                           )
    projwfc_inputs = inputs.get('projwfc', {})
    projwfc_inputs.update({"projwfc.parent_folder": nscf_node.outputs['remote_folder']})
    projwfc.set(projwfc_inputs)
    # -------- wannier90_pp -----------
    wannier90_pp = wg.nodes.new(Wannier90BaseWorkChain, name='wannier90_pp',
                                bands=nscf_node.outputs['output_band'],
    )
    wannier90_pp_inputs = inputs.get('wannier90_pp', {"wannier90": {"parameters": {}}})
    wannier90_pp_inputs.update({"wannier90.structure": structure})
    wannier90_pp.set(wannier90_pp_inputs)
    wannier90_pp_parameters = wg.nodes.new(
        prepare_wannier90_pp_inputs, name='wannier90_pp_parameters',
        parameters=wannier90_pp_inputs["wannier90"].get('parameters')
    )
    wg.links.new(nscf_node.outputs['output_parameters'], wannier90_pp_parameters.inputs['output_parameters'])
    wg.links.new(wannier90_pp_parameters.outputs[0], wannier90_pp.inputs['wannier90.parameters'])
    # -------- pw2wannier90 -----------
    pw2wannier90 = wg.nodes.new(Pw2wannier90BaseWorkChain, name='pw2wannier90',
                                bands=projwfc.outputs['bands'],
                                bands_projections=projwfc.outputs['projections'],
    )
    pw2wannier90.set({"pw2wannier90.nnkp_file": wannier90_pp.outputs['nnkp_file'],
                      "pw2wannier90.parent_folder": nscf_node.outputs['remote_folder'],
                      })
    pw2wannier90_inputs = inputs.get('pw2wannier90', {})
    pw2wannier90.set(pw2wannier90_inputs)
    # -------- wannier90 -----------
    wannier90 = wg.nodes.new(Wannier90BaseWorkChain, name='wannier90')
    wannier90_inputs = inputs.get('wannier90', {})
    wannier90_inputs.update({"wannier90.structure": structure,
                             "wannier90.remote_input_folder": pw2wannier90.outputs['remote_folder'],
                             })
    wannier90.set(wannier90_inputs)
    # export workgraph
    return wg


wannier_wg = wannier_workgraph()
wannier_wg

scf_inputs {'pw.structure': None}


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

## Prepare inputs and submit


In [16]:
from aiida import load_profile, orm
from ase.build import bulk
from copy import deepcopy
from aiida.plugins import DataFactory
load_profile()


StructureData = DataFactory('core.structure')
a = 5.68018817933178
structure = StructureData(cell = [[-a/2., 0, a/2.], [0, a/2., a/2.], [-a/2., a/2., 0]])
structure.append_atom(symbols=['Ga'], position=(0., 0., 0.))
structure.append_atom(symbols=['As'], position=(-a/4., a/4., a/4.))
structure.store()
print ( 'Structure stored with pk %d' %structure.pk)



structure = orm.StructureData(ase=bulk("Si"))
pw_code = orm.load_code("qe-7.2-pw@localhost")
projwfc_code = orm.load_code('qe-7.2-projwfc@localhost')
pw2wannier90_code = orm.load_code('qe-7.2-pw2wannier90@localhost')
wannier90_code = orm.load_code('wannier90@localhost')

scf_paras = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "smearing",
        "smearing": "cold",
        "degauss": 0.01,
    },
    "ELECTRONS": {}
}
nscf_paras = deepcopy(scf_paras)
nscf_paras['CONTROL']['calculation'] = 'nscf'
nscf_paras['SYSTEM'].update({'nosym': True, 'noinv': True})
nscf_paras['ELECTRONS'].update({'startingpot': 'file',
                                'diago_full_acc': True})

# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
scf_kpoints = orm.KpointsData()
scf_kpoints.set_kpoints_mesh([2, 2, 2])
nscf_kpoints = orm.KpointsData()
nscf_kpoints.set_kpoints_mesh([5, 5, 5])
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================

wannier_inputs = {
    'scf': {
        'pw': {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(scf_paras),
            'metadata': metadata,
        },
        'kpoints': scf_kpoints,
    },
    'nscf': {
        'pw': {
            'code': pw_code,
            'pseudos': pseudos,
            'parameters': orm.Dict(nscf_paras),
            'metadata': metadata,
        },
        'kpoints': nscf_kpoints,
    },
    'projwfc': {
        'projwfc': {
            'code': projwfc_code,
            'metadata': metadata,
        }
    },
    'wannier90_pp': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.dict({
                "band_plot": True,
                "num_wann": 4,
            }),
            'metadata': metadata,
            'settings': {'postproc_setup': True}
        },
    },
    'pw2wannier90': {
        'pw2wannier90': {
            'code': pw2wannier90_code,
            "parameters": {
                "scdm_entanglement": "erfc",
                "scdm_proj": True,
                "write_amn": True,
                "write_unk": True,
                "write_mmn": True,
            },
            'metadata': metadata,
        }
    },
    'wannier90': {
        'wannier90': {
            'code': wannier90_code,
            'parameters': orm.dict({
                "auto_projections": True,
                "band_plot": True,
                }),
            'metadata': metadata,
            'settings': {'postproc_setup': False}
        },
    },
    
}
wg = wannier_workgraph(structure=structure, inputs=wannier_inputs)
wg.name = "Wannier-Si"
wg.run()


Structure stored with pk 13041
scf_inputs {'pw': {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 687, uuid: 1a272f2f-ea8f-48a7-a1ed-cce982f322f2>, 'pseudos': {'Si': <UpfData: uuid: 945da0ed-9566-4c95-a6a4-36114036ebf8 (pk: 815)>}, 'parameters': <Dict: uuid: 7e55732c-ae69-4914-b9b0-a58c0877952b (unstored)>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}}, 'kpoints': <KpointsData: uuid: 8b009779-f4ae-49b6-877a-ad4e36a80503 (unstored)>, 'pw.structure': <StructureData: uuid: d83d8cdf-ecd8-4ccc-a390-4c41547c9a62 (unstored)>}


05/27/2024 04:59:46 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/27/2024 04:59:47 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|continue_workgraph]: nodes ready to run: scf
05/27/2024 04:59:47 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|run_nodes]: Run node: scf, type: WORKCHAIN


------------------------------------------------------------
executor:  <class 'aiida_quantumespresso.workflows.pw.base.PwBaseWorkChain'>
args:  [{'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 687, uuid: 1a272f2f-ea8f-48a7-a1ed-cce982f322f2>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 7e55732c-ae69-4914-b9b0-a58c0877952b (pk: 13042)>, 'pseudos': {'Si': <UpfData: uuid: 945da0ed-9566-4c95-a6a4-36114036ebf8 (pk: 815)>}, 'structure': <StructureData: uuid: d83d8cdf-ecd8-4ccc-a390-4c41547c9a62 (pk: 13043)>}, None, None, {}]
kwargs:  {'kpoints': <KpointsData: uuid: 8b009779-f4ae-49b6-877a-ad4e36a80503 (pk: 13044)>, 'pw': {'code': <InstalledCode: Remote code 'qe-7.2-pw' on localhost pk: 687, uuid: 1a272f2f-ea8f-48a7-a1ed-cce982f322f2>, 'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 1}}}, 'parameters': <Dict: uuid: 7e55732c-ae69-4914-b9b0-a58c0877952b (pk: 13042)

...

05/27/2024 05:00:41 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|set_node_result]: Node: projwfc finished.
05/27/2024 05:00:44 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/27/2024 05:00:45 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|continue_workgraph]: nodes ready to run: 
05/27/2024 05:00:46 PM <570398> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13047|WorkGraphEngine|is_workgraph_finished]: WorkGraph finished, but nodes: ['pw2wannier90', 'wannier90', 'wannier90_pp', 'wannier90_pp_parameters'] failed.


node:  nscf FINISHED
node:  projwfc FINISHED
node:  pw2wannier90 FAILED
node:  scf FINISHED
node:  wannier90 FAILED
node:  wannier90_pp FAILED
node:  wannier90_pp_parameters FAILED
is workgraph finished:  True


{}

# Compare with parameters with the workchain

In [15]:
from aiida_wannier90_workflows.workflows import Wannier90BandsWorkChain
from aiida import load_profile, orm
from ase.build import bulk
from copy import deepcopy
from aiida.plugins import DataFactory
load_profile()


StructureData = DataFactory('core.structure')
a = 5.68018817933178
structure = StructureData(cell = [[-a/2., 0, a/2.], [0, a/2., a/2.], [-a/2., a/2., 0]])
structure.append_atom(symbols=['Ga'], position=(0., 0., 0.))
structure.append_atom(symbols=['As'], position=(-a/4., a/4., a/4.))
structure.store()
print ( 'Structure stored with pk %d' %structure.pk)



structure = orm.StructureData(ase=bulk("Si"))
pw_code = orm.load_code("qe-7.2-pw@localhost")
projwfc_code = orm.load_code('qe-7.2-projwfc@localhost')
pw2wannier90_code = orm.load_code('qe-7.2-pw2wannier90@localhost')
wannier90_code = orm.load_code('wannier90@localhost')
codes = {
    "pw": pw_code,
    "projwfc": projwfc_code,
    "pw2wannier90": pw2wannier90_code,
    "wannier90": wannier90_code,
}
# Prepare the builder to launch the workchain.
# We use fast protocol to converge faster.
builder = Wannier90BandsWorkChain.get_builder_from_protocol(
    codes,
    structure,
    protocol="fast",
    pseudo_family="SSSP/1.3/PBEsol/efficiency",
)
builder



Structure stored with pk 13040
Summary of key input parameters:
  Formula: Si2
  PseudoFamily: SSSP/1.3/PBEsol/efficiency
  ElectronicType: METAL
  SpinType: NONE
  WannierProjectionType: SCDM
  WannierDisentanglementType: NONE
  WannierFrozenType: NONE
  num_bands: 12
  num_wann: 8
  mp_grid: [5, 5, 5]



Process class: Wannier90BandsWorkChain
Inputs:
clean_workdir: false
metadata: {}
nscf:
  kpoints: (Path of 125 kpts)
  kpoints_force_parity: false
  metadata: {}
  pw:
    code: Quantum Espresso from Conda
    metadata:
      options:
        max_wallclock_seconds: 43200
        resources:
          num_machines: 1
          num_mpiprocs_per_machine: 1
        stash: {}
        withmpi: true
    monitors: {}
    parameters:
      CONTROL:
        calculation: nscf
        etot_conv_thr: 0.0002
        forc_conv_thr: 0.001
        restart_mode: from_scratch
        tprnfor: true
        tstress: true
      ELECTRONS:
        conv_thr: 8.0e-10
        diago_full_acc: true
        electron_maxstep: 80
        mixing_beta: 0.4
        startingpot: file
      SYSTEM:
        degauss: 0.01
        ecutrho: 240.0
        ecutwfc: 30.0
        nbnd: 12
        noinv: true
        nosym: true
        occupations: smearing
        smearing: cold
    pseudos:
      Si: ''
projwfc:
  metadata: {}
